In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
SAMPLE_INDEX = 9

In [ ]:
def get_headerless_vcf_df(f_obj, stop_on='#CHROM'):
    line = ''
    header = ''
    while True:
        line = f_obj.readline()
        header += line
        if line[:6] == stop_on:
            break
            
#     print(line)
            
    df = pd.read_csv(f_obj, sep='\t', header=None)
    df.columns = line[1:].replace('""', '').replace('\n', '').split('\t')

    return df, header

In [ ]:
def get_calls_matrix(vcf_fp):
    f_obj = open(vcf_fp)
    
    line = ''
    while True:
        line = f_obj.readline()
        if line[:6] == '#CHROM':
            break
    
    line = line.replace('\n', '')
    samples = line.split('\t')[SAMPLE_INDEX:]
    
    X = []
    for line in f_obj:
        line = line.replace('\n', '')
        pieces = line.split('\t')
        X.append(pieces[SAMPLE_INDEX:])
    
    return np.asarray(X), samples

### assumptions

sample vcf is same length as genomes vcf

In [ ]:
# CALLED_VCF_FP = '/diskmnt/Projects/Users/estorrs/data/ancestry/MM/temp/called_samples.vcf'
# GENOMES_VCF_FP = '/diskmnt/Projects/Users/estorrs/1000-genomes/GRCh37/all.coding.sorted.02maf.10000sampled.sorted.snps.vcf'

In [ ]:
CALLED_VCF_FP = '/diskmnt/Projects/Users/estorrs/data/ancestry/MM/temp/mm.called_samples.vcf'
GENOMES_VCF_FP = '/diskmnt/Projects/Users/estorrs/1000-genomes/GRCh37/all.coding.sorted.02maf.10000sampled.sorted.snps.vcf'

In [ ]:
# def preprocess_vcf(vcf_fp):
#     """Pull out X, convert to 
#     Returns:
#         (X, scaler)
#     """
#     calls_matrix, samples = get_calls_matrix(vcf_fp)
    
#     for i, row in enumerate(calls_matrix):
#         row = [re.sub(re.compile(r'^0\|[1-9]+|^[1-9]+\|0'), r'0|1', v) for v in row]
#         calls_matrix[i] = [re.sub(re.compile(r'^[1-9]+\|[1-9]+'), r'1|1', v) for v in row]
        
#     # encode calls
#     label_encoder = LabelEncoder()
#     label_encoder.fit(['.|.', '0|0', '0|1', '1|1', '0', '1'])
    
#     for i, row in enumerate(calls_matrix):
#         calls_matrix[i] = label_encoder.transform(row)
    
# #     print(ca)
    
#     calls_matrix = np.transpose(calls_matrix)
    
#     return calls_matrix, samples

def get_preprocessed_X(df):    
#     if drop_indices is None:
        
    
    # remove x
    df = df[df['CHROM'] != 'X']
    df = df[df['CHROM'] != 'chrX']
    
    trimmed_df = df[df.columns[9:]]
    samples = trimmed_df.columns
    
    trimmed_df = trimmed_df.replace(re.compile(r'^0\|[1-9]+|^[1-9]+\|0'), '0|1')
    trimmed_df = trimmed_df.replace(re.compile(r'^[1-9]+\|[1-9]+'), '1|1')
    
#     return trimmed_df
    # for X chrom
#     trimmed_df = trimmed_df.replace(re.compile(r'^[1-9]+$'), '1')
    
    # encode genotype
    label_encoder = LabelEncoder()
#     label_encoder.fit(['.|.', '0|0', '0|1', '1|1', '0', '1'])
    label_encoder.fit(['.|.', '0|0', '0|1', '1|1'])

    # encode the rows
    for i, row in trimmed_df.iterrows():
        trimmed_df.loc[i][:] = label_encoder.transform(row.values)
        
    X = trimmed_df.values

    # transpose so each row is now a sample
    X = X.transpose()
    
    return X, label_encoder, samples, trimmed_df
    

In [ ]:
df_genomes, _ = get_headerless_vcf_df(open(GENOMES_VCF_FP))
df_called, _ = get_headerless_vcf_df(open(CALLED_VCF_FP))

In [ ]:
# drop rows with all missing
to_drop = []
for i, row in df_called.iterrows():
    if len([x for x in list(row[9:]) if x != '.|.']) == 0:
        to_drop.append(i)

df_genomes = df_genomes.drop(to_drop) 
df_called = df_called.drop(to_drop)

In [ ]:
X_genomes, encoder_genomes, samples_genomes, trimmed_df_genomes = get_preprocessed_X(df_genomes)

In [ ]:
X_called, encoder_called, samples_called, trimmed_df_called = get_preprocessed_X(df_called)

In [ ]:
X_genomes.shape, X_called.shape

In [ ]:
np.all(df_genomes['POS'] == df_called['POS'])

In [ ]:
trimmed_df_genomes.head(10)

In [ ]:
trimmed_df_called.head(10)

In [ ]:
# pca_scaler = StandardScaler()
# pca_scaler.fit(X_genomes)
# X_genomes = pca_scaler.transform(X_genomes)

In [ ]:
# X_called = pca_scaler.transform(X_called)

In [ ]:
print(X_genomes.shape, X_called.shape)
X = np.vstack((X_genomes, X_called))
print(X.shape)

## PCA

In [ ]:
pca_scaler = StandardScaler()
pca_scaler.fit(X)
X = pca_scaler.transform(X)

In [ ]:
pca = PCA(n_components=20)
pca.fit(X)
pcs = pca.transform(X)

In [ ]:
pcs_df = pd.DataFrame(pcs)
pd.scatter_matrix(pcs_df.loc[:, :3], figsize=(12,12))

In [ ]:
# pca = PCA(n_components=20)
# pca.fit(X_genomes)
# pcs_genomes = pca.transform(X_genomes)

In [ ]:
# pcs_genomes_df = pd.DataFrame(pcs_genomes)
# pd.scatter_matrix(pcs_genomes_df.loc[:, :3], figsize=(12,12))

In [ ]:
# pca = PCA(n_components=20)
# pca.fit(X_called)

In [ ]:
# pcs_called = pca.transform(X_called)

In [ ]:
# pcs_called_df = pd.DataFrame(pcs_called)
# pd.scatter_matrix(pcs_called_df.loc[:, :3], figsize=(12,12))

## annotate 1000 genomes

In [ ]:
# load in 1000 genome labels
labels_df = pd.read_csv('/diskmnt/Projects/Users/estorrs/1000-genomes/GRCh37/integrated_call_samples_v3.20130502.ALL.panel',
                       sep='\t')
labels_df.head()

In [ ]:
# make sure order still matches
np.all(df_genomes.columns[9:] == labels_df['sample'])

In [ ]:
sample_to_ancestry = {k:v for k, v in zip(labels_df['sample'], labels_df['super_pop'])}

ancestry_to_color = {'EUR': 'red', 'EAS': 'blue', 'AMR': 'green', 'AFR': 'yellow', 'SAS': 'pink'}
colors = [ancestry_to_color[sample_to_ancestry[s]] for s in labels_df['sample']]

In [ ]:
## extract back out into called and 1000 genomes
pcs_genomes_df = pcs_df[:X_genomes.shape[0]][:]
pcs_called_df = pcs_df[X_genomes.shape[0]:][:]
pcs_genomes_df.shape, pcs_called_df.shape

In [ ]:
def get_axis_limits(pcs_df, d=4):
    lims = []
    for i in range(d):
        c = pcs_df[i]
        lims.append((min(c), max(c)))
        
    return lims
        

In [ ]:
pd.scatter_matrix(pcs_genomes_df.loc[:, :3], color=colors, figsize=(12,12))

In [ ]:
lims = get_axis_limits(pcs_genomes_df)
lims

In [ ]:
ax = pd.plotting.scatter_matrix(pcs_called_df.loc[:, :3], figsize=(12,12), diagonal='kde')
for i, (ax_min, ax_max) in enumerate(lims):
    for j in range(len(lims)):
        ax[i, j].set_ylim(ax_min, ax_max)
        ax[j, i].set_xlim(ax_min, ax_max)

## do classifier

In [ ]:
# get target values
ancestries = labels_df['super_pop'].values
ancestry_encoder = LabelEncoder()
ancestry_encoder.fit(list(set(ancestries)))
y = ancestry_encoder.transform(ancestries)

In [ ]:
# do normalization
X = np.copy(pcs_genomes_df.values)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
rf_scaler = StandardScaler()
rf_scaler.fit(X_train)
X_train = rf_scaler.transform(X_train)
X_test = rf_scaler.transform(X_test)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

In [ ]:
# train accuracy and test accuracy
clf.score(X_train, y_train), clf.score(X_test, y_test)

## classify samples

In [ ]:
# do classification of mm samples
samples = df_called.columns[9:]

In [ ]:
samples

In [ ]:
X_samples = np.copy(pcs_called_df.values)
X_samples = rf_scaler.transform(X_samples)

In [ ]:
predictions = clf.predict(X_samples)
predictions = ancestry_encoder.inverse_transform(predictions)

In [ ]:
predictions

In [ ]:
sample_to_predictions = {k:v for k, v in zip(samples, predictions)}
colors = [ancestry_to_color[sample_to_predictions[s]] for s in samples]

In [ ]:
ax = pd.plotting.scatter_matrix(pcs_called_df.loc[:, :3], color=colors, figsize=(12,12), diagonal='kde')
for i, (ax_min, ax_max) in enumerate(lims):
    for j in range(len(lims)):
        ax[i, j].set_ylim(ax_min, ax_max)
        ax[j, i].set_xlim(ax_min, ax_max)